<a href="https://colab.research.google.com/github/piggyatbaqaqi/skol/blob/main/IST691/mistral_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
import json
from pathlib import Path
import random
from typing import Any, Dict, List, Optional
import glob
from google.colab import drive
%cd /content
content = Path('/content')
skol = content / 'drive/My Drive/SKOL'
piggyatbaqaqi = skol / 'github.com/piggyatbaqaqi'
drive.mount(str(content / "drive"), force_remount=True)
skol_client = content / 'skol'
if not os.path.exists(skol_client):
  skol_client.symlink_to(piggyatbaqaqi / 'skol')
ist691 = skol_client / 'IST691'

sys.path.insert(0, str(piggyatbaqaqi / 'skol'))

/content
Mounted at /content/drive


In [2]:
# You only need to run this once per machine
try:
  import bitsandbytes
except ImportError:
  !pip install bitsandbytes
try:
  import transformers
except ImportError:
  !pip install transformers
try:
  import peft
except ImportError:
  !pip install peft
try:
  import accelerate
except ImportError:
  !pip install accelerate
try:
  import datasets
  import ipywidgets
  import fsspec
except ImportError:
  !pip install datasets scipy ipywidgets
  !pip install fsspec==2023.9.2


### Set up git clients

In [3]:
if not os.path.exists(piggyatbaqaqi):
  %mkdir -p $piggyatbaqaqi
if not os.path.exists(piggyatbaqaqi / 'skol'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/skol.git
sys.path.insert(0, piggyatbaqaqi / 'skol')
if not os.path.exists(piggyatbaqaqi / 'dr-drafts-mycosearch'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/dr-drafts-mycosearch.git
workdir = skol / 'IST691'
%cd $workdir

/content/drive/My Drive/SKOL/IST691


In [4]:
prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.
'''
# prompt = '''Format the output as JSON.
# Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
# The innermost layer must be formated as lists of string-valued values..
# Translate. if any, Latin paragraphs to English.
# '''

eval_description = """Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicalis rotundata;
cellula basalis cylindrica vel conico-truncata, ad basim 3.5–4.5 μm crassa; Appendicibus
lateralibus 0–2, brunneae, septata, cylindricae, surgentibus ex cellulla e apicem 2nd vel
3rd.

Anamorphic fungi. Colonies on natural substrate eﬀuse, black. Mycelium
superﬁcial, composed of branched, septate, pale brown to brown, smoothwalled hyphae, 1.5–3 μm thick. Conidiophores absent or short, 1–3-septate,
brown to dark brown, 11–28 × 4.5–5 μm. Conidiogenous cells monoblastic,
determinate, solitary, simple, lageniform or ampulliform, brown to dark brown,
smooth, 4.5–6.5 × 3.5–5 μm, 3–4.5 μm wide at the truncate apex. Conidial
secession schizolytic. Conidia holoblastic, solitary, acrogenous, straight or
curved, obclavate to obclavate-rostrate, dark brown to brown, smooth, 13–19distoseptate, 130–190 μm long, 7–9 μm thick in the broadest part, tapering
to 2–3 μm near the apex; apical cells rounded; basal cell cylindrical, truncate,
3.5–4.5 μm wide; lateral appendages 0–2, brown, septate, cylindrical, arising
from the 2nd or 3rd cells from the apex.
"""

### Set up SKOL-specific code

In [5]:
from finder import read_files, parse_annotated, target_classes
from label import Label
from taxon import Taxon, group_paragraphs

SEED=12345
default_label = Label('Misc-exposition')
keep_labels = [Label('Description'), Label('Nomenclature')]

In [6]:
raw_directory_path = skol / 'raw_2025_02_05/'
ann_directory_path = skol / 'annotated_2025_02_27/journals'

In [33]:
def make_prompt(prompt: str, description: str, result: Optional[str] = None) -> str:
  retval = f"""<s>[INST]{prompt}

Here is the description:
{description}[/INST]

Result:
"""
  if result is not None:
    retval += f"""
```json
{result}
```
</s>
"""
  return retval

In [8]:
# Function that reports all the txt files under a Google Drive folder path
def listFiles(folder: str) -> List[str]:
  # List all files in the folder
  try:
      files = [file for file in glob.glob(f'{folder}/**/*.txt*', recursive=True) if 'Sydowia' not in file]
      return files
  except FileNotFoundError:
      print(f"Folder '{folder}' not found.")
  except PermissionError:
      print(f"Permission denied to access folder '{folder}'.")

In [9]:
# check files in annotated directory
training_files = listFiles(ann_directory_path)
training_files[:10]

['/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol057/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol054/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s17.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s29.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s30.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s7.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s21.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s13.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s46.txt.ann']

In [10]:
paragraphs = list(parse_annotated(read_files(random.sample(training_files, 100))))
relabeled = list(target_classes(default=default_label, keep=keep_labels, paragraphs=paragraphs))

In [11]:
def load_json_training(filename: str) -> List[Dict[str, Any]]:
  retval = []
  def val(description: str, result: Dict[str, Any]) -> Dict[str, str]:
    return {'description': description,
            'result': json.dumps(result, indent=4, ensure_ascii=False)}
  state = 'START'  # 'description', 'result'
  with open(filename, "r", encoding="utf-8") as file:
    lines = []
    description = ''
    for line in file:
      if line.startswith('Send to LLM:'):
        if state == "result":
          result = ''.join(lines)
          try:
            result_dict = json.loads(result)
          except json.JSONDecodeError as err:
            print(f'Err: {err}\n{result}')

          retval.append(val(description, result_dict))
        lines = []
        state = 'description'
      elif line.startswith('Result:'):
        if state == "description":
          description = ''.join(lines)
          lines = []
        state = 'result'
      else:
        lines.append(line)
    if state == 'result' and len(lines) > 0:
      result = ''.join(lines)
      try:
        result_dict = json.loads(result)
      except json.JSONDecodeError as err:
        print(f'Err: {err}\n{result}')
      retval.append(val(description, result_dict))
  return retval


In [12]:
json_training = load_json_training(workdir / 'json_training.txt')
print(json_training[0])

{'description': '\n\nSaprobic on stems. Ascomata, superﬁcial, thyriothecial; in section conical,\nrelatively small, opening with a minute ﬂat or slightly papillate ostiole. Asci 8spored, bitunicate, ﬁssitunicate, cylindrical or obclavate. Ascospores, 1-septate,\nlight brown.\nAnamorphs: None reported for the genus (Hyde et al. 2011).\n\n\n', 'result': '{\n    "habit": [\n        "saprobic"\n    ],\n    "habitat": [\n        "on stems"\n    ],\n    "ascomata": [\n        "superficial",\n        "thyriothecial",\n        "conical",\n        "relatively small",\n        "opening with a minute flat or slightly papillate ostiole"\n    ],\n    "asci": {\n        "number of spores": [\n            "8-spored"\n        ],\n        "shape": [\n            "bitunicate",\n            "fissitunicate",\n            "cylindrical or obclavate"\n        ]\n    },\n    "ascospores": {\n        "septation": [\n            "1-septate"\n        ],\n        "color": [\n            "light brown"\n        ]\n

In [13]:
import datasets

dataset = datasets.Dataset.from_list(json_training)

new_dataset = datasets.Dataset.train_test_split(dataset, int(1), shuffle=False)
temp_dataset = new_dataset["train"]
test_dataset = new_dataset["test"]
new_dataset2 = datasets.Dataset.train_test_split(temp_dataset,int(1), shuffle=False)
train_dataset = new_dataset2["train"]
eval_dataset = new_dataset2["test"]

print(train_dataset, eval_dataset, test_dataset)

Dataset({
    features: ['description', 'result'],
    num_rows: 14
}) Dataset({
    features: ['description', 'result'],
    num_rows: 1
}) Dataset({
    features: ['description', 'result'],
    num_rows: 1
})


In [14]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
from google.colab import userdata
token = userdata.get('HF_TOKEN')

login(token=token)
base_model_id = "mistralai/Mistral-7B-Instruct-v0.3"
max_length = 2048

In [16]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=max_length,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token


In [18]:

def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = make_prompt(prompt, data_point["description"], data_point["result"])
    return tokenize(full_prompt)


In [20]:

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)
tokenized_test_dataset = test_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [21]:
print(tokenized_train_dataset[4]['input_ids'])
print(len(tokenized_train_dataset[4]['input_ids']))
print("\n" + test_dataset[0]['description'])
print("Result: \n```json\n" + test_dataset[0]['result'] + "\n```\n")


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [22]:
eval_prompt = make_prompt(prompt, eval_description)


In [23]:
print(eval_prompt)

<s>[IST]Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.


Here is the description:
Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.

In [24]:
base_model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True))


[IST]Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.


Here is the description:
Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μ

## Run the base model on an assortment of annotated files

In [25]:
grouped = group_paragraphs(relabeled)
with open(workdir / 'json_training3.txt', 'w', encoding='utf-8') as f:
  for i, tax in enumerate(grouped):
    eval_prompt = make_prompt(prompt, tax.as_row()["description"])
    base_model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    base_model.eval()
    print(tax.as_row()["taxon"])
    with torch.no_grad():
        print(tokenizer.decode(base_model.generate(**base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True), file=f)


Xenodochus carbonarius Schltdl., Linnaea 1: 237 (1826)

Puccinia hysteriiformis Peck, Bot. Gaz. 6: 276 (1881)

Puccinia majoricensis Maire, Bull. Soc. Myc. Fr. 21: 221 (1905)

Puccinia pedunculata J.Schröt., Jber. schles. Ges. vaterl. Kultur 52: 117 (1875)

Puccinia stachydis DC., Fl. franc. 2: 595 (1805)

Puccinia tendae Gäum., Rev. Mycol. 18: 184 (1953)

Uromyces colchici Massee, Grevillea 21: 6 (1892)

Uromyces kochiae Syd. & P. Syd., Annls mycol. 10: 215 (1912)

Uromyces veratri (DC.) J. Schröt., Jber. schles. Ges. vaterl. Kultur 49: 10 (1871)

Fomitiporia apiahyna (Speg.) Vlasák & Kout, comb. nov.

≡ Fomes apiahynus Speg., Bol. Acad. Nac. Ci. 11: 438, 1889.

Caloplaca allochroa Y. Joshi, Vondrák & Hur, sp. nov.

1, STRIGULA LATERALIS Aptroot & v. d. Boom spec. nov. Fig. 1

Ellisembia adscendens (Berk.) Subram., Proceedings of the Indian National
Science Academy B 58: 183, 1992 [Fig. 1]

Sporidesmium adscendens Berk., Annals of Natural History 4: 291, 1840

Ellisembia brachypus (El

Token indices sequence length is longer than the specified maximum sequence length for this model (2474 > 2048). Running this sequence through the model will result in indexing errors


Rhizomucor endophyticus Zheng & H. Jiang sp. nov. (Figs. 1-3; Pls. I)

Hyphodontiella hauerslevii K.H. Larss. & Hjortstam nov. sp.

Oncopodiella hungarica sp. nov. (Fig.l. A)

Oncopodiella robusta sp.nov. (Fig.l. B)

≡ Volvariella nigrovolvacea var. dunensis Vila, Àngel & Llimona,
Revista Catalana de Micologia 22: 131. 1999.

Fuscoporia palomari Vlasák & Ryvarden, sp. nov.

Antrodia madronae Vlasák & Ryvarden, sp. nov.

Skeletocutis subodora Vlasák & Ryvarden, sp. nov.

Queletia mirabilis Fr., Öfvers. Förhandl. K. Svenska Vetensk.-Akad. 28: 171 (1872)

Helvella fusca Gillet, Champignons de France, Discom.: 9, pl. 4, 1879.

= Helvella fusca var. bresadolae Boud., Icon. Mycol. (Paris),
Tome 2: pl. 230, 1910; Tome 4: 121, 1911.

= Helvella fusca var. gyromitroides Chenant., in Pelé & Chenantais,
Bull. Soc. Sci. Nat. Ouest, Sér. 4, 1: 75, 1921.

≠ Helvella sulcata α fusca Afzel., K. Vetensk.-Acad. Handl. 4: 305, Tab. X, ﬁg. 1, 1783.

Cortinarius callimorphus Bojantchev & R.M. Davis, sp. no

In [26]:
from peft import prepare_model_for_kbit_training
base_model.gradient_checkpointing_enable()
base_model = prepare_model_for_kbit_training(base_model)
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )




In [27]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)
base_model = get_peft_model(base_model, config)
print_trainable_parameters(base_model)
base_model = accelerator.prepare_model(base_model)



trainable params: 21266432 || all params: 3779629056 || trainable%: 0.5626592367904582


In [28]:
print(base_model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [29]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    base_model.is_parallelizable = True
    base_model.model_parallel = True


In [30]:
!nvidia-smi

Thu Jun 19 14:12:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             57W /  400W |    7627MiB /  40960MiB |      1%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Fine tune the model

In [ ]:
import transformers
from datetime import datetime
import wandb
project = "skol-finetune"
num_steps = 1000
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name
tokenizer.pad_token = tokenizer.eos_token
wandb_token = userdata.get('WANDB_TOKEN')
wandb.login(key=wandb_token)
trainer = transformers.Trainer(
    model=base_model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # max_steps=1000,
        max_steps=num_steps,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=10,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=10,                # Save checkpoints every 50 steps
        eval_strategy="steps", # Evaluate the model every logging step
        eval_steps=10,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
base_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
10,0.065100,1.178788
20,0.043500,1.206740
30,0.037700,1.211330
40,0.034800,1.206184
50,0.031300,1.246074
60,0.028800,1.268315
70,0.026500,1.268483
80,0.024500,1.259060
90,0.022100,1.266927
100,0.020100,1.247529


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embeddi

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
base_tokenizer.pad_token = tokenizer.eos_token


In [ ]:
print(eval_prompt)
print(prompt)

In [ ]:
eval_prompt = make_prompt(prompt, eval_description)

In [ ]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, f"mistral-skol-finetune/checkpoint-{num_steps}")
ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True))


## Save the finetuned model and tokenizer


In [ ]:
ft_model.save_pretrained(str(ist691 / 'mistral-skol-finetune'))
tokenizer.save_pretrained(str(ist691 / 'mistral-skol-finetune'))

## Prompt engineering cell
Currently disabled.

manual_prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.
'''

manual_description = """Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicalis rotundata;
cellula basalis cylindrica vel conico-truncata, ad basim 3.5–4.5 μm crassa; Appendicibus
lateralibus 0–2, brunneae, septata, cylindricae, surgentibus ex cellulla e apicem 2nd vel
3rd.

Anamorphic fungi. Colonies on natural substrate eﬀuse, black. Mycelium
superﬁcial, composed of branched, septate, pale brown to brown, smoothwalled hyphae, 1.5–3 μ
m thick. Conidiophores absent or short, 1–3-septate,
brown to dark brown, 11–28 × 4.5–5 μm. Conidiogenous cells monoblastic,
determinate, solitary, simple, lageniform or ampulliform, brown to dark brown,
smooth, 4.5–6.5 × 3.5–5 μm, 3–4.5 μm wide at the truncate apex. Conidial
secession schizolytic. Conidia holoblastic, solitary, acrogenous, straight or
curved, obclavate to obclavate-rostrate, dark brown to brown, smooth, 13–19distoseptate, 130–190 μm long, 7–9 μm thick in the broadest part, tapering
to 2–3 μm near the apex; apical cells rounded; basal cell cylindrical, truncate,
3.5–4.5 μm wide; lateral appendages 0–2, brown, septate, cylindrical, arising
from the 2nd or 3rd cells from the apex.
"""

manual_eval_prompt = make_prompt(manual_prompt, manual_description)


manual_base_model_input = tokenizer(manual_eval_prompt, return_tensors="pt").to("cuda")
base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**manual_base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True))


In [ ]:
import json
import numpy as np
from numpy.linalg import norm

def key_value_sets(json_obj):
  keys = set()
  values = set()
  if isinstance(json_obj, dict):
    for key, value in json_obj.items():
      keys.add(key)
      subkeys, subvalues = key_value_sets(value)
      keys.update(subkeys)
      values.update(subvalues)
  elif isinstance(json_obj, list):
    values.update(json_obj)
  elif isinstance(json_obj, str):
    values.add(json_obj)
  else:
    raise ValueError(f"Not a valid type:{type(json_obj)}-#{json_obj}")
  return (keys,values)

# Credit to Google search of "jaccard distance set python" which generated this code.
def jaccard_distance(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard_index = len(intersection) / len(union)
    jaccard_distance = 1 - jaccard_index
    return jaccard_distance

def jaccard_distance_json(json1, json2):
    """
    Calculates the jaccard distance between two JSON objects.

    We compare the set of keys and subkeys between the two JSON objects, and
    the set of values between the two JSON objects, and return the average of
    the jaccard distance between the two sets.

    Args:
        json1 (dict): The first JSON object.
        json2 (dict): The second JSON object.

    Returns:
        float: The jaccard distance between the two JSON objects.
    """

    json1_keys, json1_vals = key_value_sets(json1)
    json2_keys, json2_vals = key_value_sets(json2)
    j_key = jaccard_distance(json1_keys, json2_keys)
    j_val = jaccard_distance(json1_vals, json2_vals)

    avg_jaccard_distance = (j_key + j_val)/2

    return avg_jaccard_distance

# Example Usage
json_training_0 = json.loads(json_training[0]['result'])
json_training_1 = json.loads(json_training[1]['result'])
#print(json_training[0]['result'])
#print(json_training[1]["result"])
print(jaccard_distance_json(json_training_0, json_training_0))
print(jaccard_distance_json(json_training_0, json_training_1))


In [ ]:
import io

def extract_json(md: str) -> Dict[str, Any]:
  """
  Extracts the first JSON object from a string of Markdown.
  """
  state = "START"
  lines = []
  with io.StringIO(md) as f:
    for line in f:
      if line.startswith('```json') or line.startswith("result:"):
        state = "RECORDING"
      elif line.startswith('```'):
        state = "END"
        return json.loads("\n".join(lines))
      elif state == "RECORDING":
        lines.append(line)
  return json.loads("\n".join(lines))



In [ ]:
test_description = test_dataset[0]['description']
ground_truth = json.loads(test_dataset[0]['result'].lower())
test_prompt = make_prompt(prompt, test_description)
print(f'test_prompt:\n{test_prompt}')

In [ ]:
base_model_input = tokenizer(test_prompt, return_tensors="pt").to("cuda")
base_model.eval()
base_model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)
base_model.eval()

In [ ]:
ft_path = str(ist691 / 'mistral-skol-finetune')
ft_tokenizer = AutoTokenizer.from_pretrained(ft_path, trust_remote_code=True)
ft_model_input = ft_tokenizer(test_prompt, return_tensors="pt").to("cuda")
ft_model = AutoModelForCausalLM.from_pretrained(ft_path, quantization_config=bnb_config)
ft_model.eval()

In [ ]:
with torch.no_grad():
  base_model_output = tokenizer.decode(base_model.generate(**base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True).lower()
  ft_model_output = tokenizer.decode(ft_model.generate(**ft_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True).lower()
  print(f'base_model_output:\n{base_model_output}')
  print(f'ft_model_output:\n{ft_model_output}')
  base_model_result = extract_json(base_model_output)
  ft_model_result = extract_json(ft_model_output)

In [ ]:
with torch.no_grad():
  base_model_output = tokenizer.decode(base_model.generate(**base_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True).lower()
  ft_model_output = tokenizer.decode(ft_model.generate(**ft_model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True).lower()
  print(f'base_model_output:\n{base_model_output}')
  print(f'ft_model_output:\n{ft_model_output}')

In [ ]:
base_model_result = extract_json(base_model_output)
ft_model_result = extract_json(ft_model_output)

In [ ]:
print("Base Model Result")
print(type(base_model_result))
print(base_model_result)
print("Fine-Tuned Model Result")
print(type(ft_model_result))
print(ft_model_result)
print("Ground Truth")
print(type(ground_truth))
print(ground_truth)

In [ ]:
print("Base Model Jaccard Distance")
print(jaccard_distance_json(base_model_result, ground_truth))
print("Fine-Tuned Model Jaccard Distance")
print(jaccard_distance_json(ft_model_result, ground_truth))
#